**Source:** [Estimating a probability distribution non-parametrically with a kernel density estimation](https://ipython-books.github.io/76-estimating-a-probability-distribution-nonparametrically-with-a-kernel-density-estimation/) ( Uses `scipy.stats.gaussian_kde`)

In [ ]:
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KernelDensity

**Original article uses a weather dataset: it contains information about most storms since 1848. A single storm may appear multiple times across several consecutive days.**

In [ ]:
# # www.ncdc.noaa.gov/ibtracs/index.php?name=wmo-data
# url = "https://raw.githubusercontent.com/ipython-books/cookbook-2nd-data/master/Allstorms.ibtracs_wmo.v03r05.csv"
# df = pd.read_csv(url)
# df[df.columns[[0, 1, 3, 8, 9]]].head()

In [ ]:
# We use pandas groupby() function to obtain the average location of every storm
# dfs = df.groupby('Serial_Num')
# pos = dfs[['Latitude', 'Longitude']].mean()
# x = pos.Longitude.values
# y = pos.Latitude.values
# pos.head()

## Coordinates of samples from Pangaea
We will adapt the code for plotting our image sample locations

In [ ]:
# Load files
DATA_DIR = "../query-outputs/"
files = os.listdir(DATA_DIR)
df_list = [pd.read_csv(os.path.join(DATA_DIR, f)) for f in files]
print(f"Total {len(df_list)} files loaded.")

# Check if all files have lat, lon data
def has_lat_lon(frame):
    cond1 = any([col == "Latitude" for col in frame.columns])
    cond2 = any([col == "Longitude" for col in frame.columns])
    return cond1 and cond2


print("All files have lat/lon columns:", all([has_lat_lon(df) for df in df_list]))
# Join all datasets
all_dfs = pd.concat(df_list)

# Values for plotting
x = all_dfs["Longitude"].dropna().to_numpy()
y = all_dfs["Latitude"].dropna().to_numpy()
print("x.shape:", x.shape, "y.shape:", y.shape)

## 1. Scatter plot

In [ ]:
# Transform
transform = ccrs.Geodetic()
# Projection
projection = ccrs.EqualEarth()

# We create the map plot.
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)

# We display the world map picture.
ax.stock_img()
# We display the storm locations.
ax.scatter(x, y, color="r", alpha=0.15, transform=transform)
ax.set_title("Spatial distribution of image samples")
plt.show()

In [ ]:
# There's no need to use 180k datapoints for prototyping. Let's downsample them.
# Check where the subsamples are placed, if we take every 200th datapoint.

# We create the map plot.
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)

# We display the world map picture.
ax.stock_img()
# We display the storm locations.
ax.scatter(x[::200], y[::200], color="r", alpha=0.15, transform=transform)
ax.set_title("Spatial distribution of image samples")
plt.show()

## 2. KDE SciPy

**Transform the image positions from the geodetic coordinate system (longitude and latitude) into the map's coordinate system**

In [ ]:
geo = ccrs.Geodetic()
h = projection.transform_points(geo, x, y)[:, :2].T
h.shape

**Now, we perform the kernel density estimation on our (2, N) array.** using `scipy.stats.gaussian_kde`

In [ ]:
kde = st.gaussian_kde(h)

**The `gaussian_kde()` routine returned a Python function. To see the results on a map, we need to evaluate this function on a 2D grid spanning the entire map. We create this grid with `meshgrid()`, and we pass the x and y values to the `kde()` function:**

In [ ]:
k = 100
# Coordinates of the four corners of the map.
x0, x1, y0, y1 = ax.get_extent()
# We create the grid.
tx, ty = np.meshgrid(np.linspace(x0, x1, 2 * k), np.linspace(y0, y1, k))
# We reshape the grid for the kde() function.
mesh = np.vstack((tx.ravel(), ty.ravel()))
# We evaluate the kde() function on the grid.
v = kde(mesh).reshape((k, 2 * k))

**Before displaying the KDE heatmap on the map, we need to use a special colormap with a transparent channel. This will allow us to superimpose the heatmap on the stock image:**

In [ ]:
# https://stackoverflow.com/a/37334212/1595060
cmap = plt.get_cmap("Reds")
my_cmap = cmap(np.arange(cmap.N))
my_cmap[:, -1] = np.linspace(0, 1, cmap.N)
my_cmap = ListedColormap(my_cmap)

**Finally, we display the estimated density with `imshow()` or `contourf()`:**

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()

# ax.imshow(v, origin='lower', cmap=my_cmap, extent=[x0, x1, y0, y1], interpolation='bilinear')
ax.contourf(
    tx,
    ty,
    v,
    cmap=my_cmap,
    extent=[x0, x1, y0, y1],
    levels=np.linspace(0, v.max(), 25),
    origin="lower",
)
# ax.scatter(x, y, color='r', s=5, alpha=0.15, transform=transform)
ax.set_title("KDE of image sample spatial distribution")
plt.show()

### Side by side

In [ ]:
fig = plt.figure(figsize=(25, 8))

# KDE plot
ax = fig.add_subplot(121, projection=projection)
ax.stock_img()
ax.set_title("KDE plot", fontsize=20)
ax.contourf(
    tx,
    ty,
    v,
    cmap=my_cmap,
    levels=np.linspace(0, v.max(), 25),
    origin="lower",
    extent=[x0, x1, y0, y1],
    interpolation="bilinear",
)
# Scatter plot
ax = fig.add_subplot(122, projection=projection)
ax.stock_img()
ax.set_title("Scatter plot", fontsize=20)
ax.scatter(x, y, color="r", alpha=0.25, transform=transform)

## 3. KDE Sklearn
The function below is based on the `kde2()` function from the following article: [Two-dimensional kernel density estimate: comparing scikit-learn and scipy](https://gist.github.com/daleroberts/7a13afed55f3e2388865b0ec94cd80d2)

In [ ]:
def kde_sklearn(
    x,
    y,
    metric="euclidean",
    bw="silverman",
    bw_factor=1.0,
    return_scores=True,
    full_grid=True,
    n_grid=100,
):
    if metric == "haversine":
        lon = x
        lat = y
        x = np.radians(lat)
        y = np.radians(lon)
    xy = np.stack([x, y], axis=-1)
    # Bandwidth calculation
    n_samp, n_feat = xy.shape
    if isinstance(bw, float):
        pass
    elif not isinstance(bw, str):
        raise ValueError(
            f"bw must be a float or a string, but {bw.__class__} instance was given"
        )
    elif bw.lower() == "silverman":
        bw = (n_samp * (n_feat + 2) / 4.0) ** (-1.0 / (n_feat + 4))  # silverman
    elif bw.lower() == "scott":
        bw = n_samp ** (-1.0 / (n_feat + 4))  # scott
    else:
        raise ValueError(f"Unsupported bandwidth estimator: {bw}")
    bw *= bw_factor
    print(f"bw: {bw}, metric: {metric}")
    # KDE
    kde = KernelDensity(
        bandwidth=bw,
        metric=metric,
        kernel="gaussian",
        algorithm="ball_tree",
    )
    kde.fit(xy)
    # Extent
    if not full_grid:
        xmin = x.min()
        xmax = x.max()
        ymin = y.min()
        ymax = y.max()
    elif metric == "haversine":
        xmin = -np.pi
        xmax = np.pi
        ymin = -np.pi / 2
        ymax = np.pi / 2
    else:
        xmin = -180
        xmax = 180
        ymin = -90
        ymax = 90
    # Mesh grid
    X, Y = np.meshgrid(
        np.linspace(xmin, xmax, n_grid * 2), np.linspace(ymin, ymax, n_grid)
    )
    positions = np.stack([X.ravel(), Y.ravel()], axis=-1)
    if metric == "haversine":
        positions = positions[:, ::-1]
    # Z heights
    Z = np.reshape(kde.score_samples(positions), X.shape)
    if not return_scores:
        Z = np.exp(Z)
    return X, Y, Z

In [ ]:
X, Y, Z = kde_sklearn(x, y, metric="euclidean")

In [ ]:
X2_, Y2_, Z2_ = kde_sklearn(x[::100], y[::100], metric="haversine", bw_factor=0.035)

In [ ]:
# We create the map plot.
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)

# We display the world map picture.
ax.stock_img()
# We display the sample locations.
ax.scatter(np.degrees(X2_), np.degrees(Y2_), color="r", alpha=0.1, transform=transform)
ax.set_title("Spatial distribution of grid samples")
plt.show()

In [ ]:
x0, x1, y0, y1

In [ ]:
# Changing transform to ccrs.PlateCarree() has no effect

# We create the map plot.
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)

# We display the world map picture.
ax.stock_img()
# We display the sample locations.
ax.scatter(
    np.degrees(X2_),
    np.degrees(Y2_),
    color="r",
    alpha=0.15,
    transform=ccrs.PlateCarree(),
)
ax.set_title("Spatial distribution of grid samples")
plt.show()

In [ ]:
Z2_

In [ ]:
# Use scatter plot to indicate KDE density by using the point estimate
# of the density as the size of the scatter dot

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.scatter(
    np.degrees(X2_),
    np.degrees(Y2_),
    s=Z2_,
    color="r",
    transform=ccrs.PlateCarree(),
)
ax.set_title("KDE of image sample spatial distribution")
plt.show()

We appear to have some errant data points in a stripe across America at one particular longitude, for all latitudes?

In [ ]:
plt.figure(figsize=(16, 7))
plt.hist(x, 100)
plt.xlim([-180, 180])
plt.show()

In [ ]:
plt.figure(figsize=(16, 7))
plt.hist(y, 100)
plt.xlim([-90, 90])
plt.show()

In [ ]:
plt.figure(figsize=(16, 7))
plt.hist(x, 1000)
plt.xlim([-180, 180])
plt.show()

In [ ]:
xu, xc = np.unique(x, return_counts=True)
idx = np.argsort(xc)[::-1]

for i in idx[:40]:
    print(f"At latitude {xu[i]:13.9f} there are{xc[i]:6d} samples")

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    Z2_,
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    # levels=np.linspace(0, Z2_.max(), 25),
    interpolation="bilinear",
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution")
plt.show()

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    Z2_,
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(0, Z2_.max(), 25),
    interpolation="bilinear",
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution")
plt.show()

In [ ]:
X2_, Y2_, Z2_ = kde_sklearn(
    x[::100], y[::100], metric="haversine", bw_factor=0.001, n_grid=200
)


fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.scatter(
    np.degrees(X2_),
    np.degrees(Y2_),
    color="r",
    s=Z2_,
    transform=ccrs.PlateCarree(),
)
ax.set_title("KDE of image sample spatial distribution (grid scatter)")
plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    Z2_,
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(Z2_.min(), Z2_.max(), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (scores)")
plt.show()

if Z2_.max() > 0:
    fig = plt.figure(figsize=(25, 8))
    ax = fig.add_subplot(projection=projection)
    ax.stock_img()
    ax.contourf(
        np.degrees(X2_),
        np.degrees(Y2_),
        Z2_,
        cmap=my_cmap,
        # extent=[x0, x1, y0, y1],
        levels=np.linspace(np.minimum(0, Z2_.min()), Z2_.max(), 25),
        # origin='lower',
        # transform=transform,
        transform=ccrs.PlateCarree(),
        # transform=ccrs.RotatedPole(),
    )
    # ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
    ax.set_title("KDE of image sample spatial distribution (scores > 0)")
    plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    np.exp(Z2_),
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(0, np.exp(Z2_.max()), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (densities)")
plt.show()

In [ ]:
X2_, Y2_, Z2_ = kde_sklearn(x[::100], y[::100], metric="haversine", bw_factor=0.01)


fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.scatter(
    np.degrees(X2_),
    np.degrees(Y2_),
    color="r",
    s=Z2_,
    transform=ccrs.PlateCarree(),
)
ax.set_title("KDE of image sample spatial distribution (grid scatter)")
plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    Z2_,
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(Z2_.min(), Z2_.max(), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (scores)")
plt.show()

if Z2_.max() > 0:
    fig = plt.figure(figsize=(25, 8))
    ax = fig.add_subplot(projection=projection)
    ax.stock_img()
    ax.contourf(
        np.degrees(X2_),
        np.degrees(Y2_),
        Z2_,
        cmap=my_cmap,
        # extent=[x0, x1, y0, y1],
        levels=np.linspace(np.minimum(0, Z2_.min()), Z2_.max(), 25),
        # origin='lower',
        # transform=transform,
        transform=ccrs.PlateCarree(),
        # transform=ccrs.RotatedPole(),
    )
    # ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
    ax.set_title("KDE of image sample spatial distribution (scores > 0)")
    plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    np.exp(Z2_),
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(0, np.exp(Z2_.max()), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (densities)")
plt.show()

In [ ]:
X2_, Y2_, Z2_ = kde_sklearn(x[::100], y[::100], metric="haversine", bw_factor=0.1)


fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.scatter(
    np.degrees(X2_),
    np.degrees(Y2_),
    color="r",
    s=Z2_,
    transform=ccrs.PlateCarree(),
)
ax.set_title("KDE of image sample spatial distribution (grid scatter)")
plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    Z2_,
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(Z2_.min(), Z2_.max(), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (scores)")
plt.show()

if Z2_.max() > 0:
    fig = plt.figure(figsize=(25, 8))
    ax = fig.add_subplot(projection=projection)
    ax.stock_img()
    ax.contourf(
        np.degrees(X2_),
        np.degrees(Y2_),
        Z2_,
        cmap=my_cmap,
        # extent=[x0, x1, y0, y1],
        levels=np.linspace(np.minimum(0, Z2_.min()), Z2_.max(), 25),
        # origin='lower',
        # transform=transform,
        transform=ccrs.PlateCarree(),
        # transform=ccrs.RotatedPole(),
    )
    # ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
    ax.set_title("KDE of image sample spatial distribution (scores > 0)")
    plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    np.exp(Z2_),
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(0, np.exp(Z2_.max()), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (densities)")
plt.show()

In [ ]:
X2_, Y2_, Z2_ = kde_sklearn(x[::100], y[::100], metric="haversine", bw_factor=1.0)


fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.scatter(
    np.degrees(X2_),
    np.degrees(Y2_),
    color="r",
    s=Z2_,
    transform=ccrs.PlateCarree(),
)
ax.set_title("KDE of image sample spatial distribution (grid scatter)")
plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    Z2_,
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(Z2_.min(), Z2_.max(), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (scores)")
plt.show()

if Z2_.max() > 0:
    fig = plt.figure(figsize=(25, 8))
    ax = fig.add_subplot(projection=projection)
    ax.stock_img()
    ax.contourf(
        np.degrees(X2_),
        np.degrees(Y2_),
        Z2_,
        cmap=my_cmap,
        # extent=[x0, x1, y0, y1],
        levels=np.linspace(np.minimum(0, Z2_.min()), Z2_.max(), 25),
        # origin='lower',
        # transform=transform,
        transform=ccrs.PlateCarree(),
        # transform=ccrs.RotatedPole(),
    )
    # ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
    ax.set_title("KDE of image sample spatial distribution (scores > 0)")
    plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    np.exp(Z2_),
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(0, np.exp(Z2_.max()), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (densities)")
plt.show()

In [ ]:
X2_, Y2_, Z2_ = kde_sklearn(x[::100], y[::100], metric="haversine", bw_factor=10.0)


fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.scatter(
    np.degrees(X2_),
    np.degrees(Y2_),
    color="r",
    s=Z2_,
    transform=ccrs.PlateCarree(),
)
ax.set_title("KDE of image sample spatial distribution (grid scatter)")
plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    Z2_,
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(np.minimum(0, Z2_.min()), Z2_.max(), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (scores)")
plt.show()

if Z2_.max() > 0:
    fig = plt.figure(figsize=(25, 8))
    ax = fig.add_subplot(projection=projection)
    ax.stock_img()
    ax.contourf(
        np.degrees(X2_),
        np.degrees(Y2_),
        Z2_,
        cmap=my_cmap,
        # extent=[x0, x1, y0, y1],
        levels=np.linspace(np.minimum(0, Z2_.min()), Z2_.max(), 25),
        # origin='lower',
        # transform=transform,
        transform=ccrs.PlateCarree(),
        # transform=ccrs.RotatedPole(),
    )
    # ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
    ax.set_title("KDE of image sample spatial distribution (scores > 0)")
    plt.show()

fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.contourf(
    np.degrees(X2_),
    np.degrees(Y2_),
    np.exp(Z2_),
    cmap=my_cmap,
    # extent=[x0, x1, y0, y1],
    levels=np.linspace(0, np.exp(Z2_.max()), 25),
    # origin='lower',
    # transform=transform,
    transform=ccrs.PlateCarree(),
    # transform=ccrs.RotatedPole(),
)
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution (densities)")
plt.show()

**euclidean**

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.imshow(
    Z, cmap=my_cmap, extent=[x0, x1, y0, y1], interpolation="bilinear", origin="lower"
)
# ax.contourf(X, Y, Z, cmap=my_cmap, extent=[x0, x1, y0, y1],
#             levels=np.linspace(0, v.max(), 25),
#             interpolation='bilinear',
#             origin='lower')
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution")
plt.show()

**haversine**

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=projection)
ax.stock_img()
ax.imshow(
    Z2_, cmap=my_cmap, extent=[x0, x1, y0, y1], interpolation="bilinear", origin="lower"
)
# ax.contourf(X2, Y2, Z2, cmap=my_cmap, extent=[x0, x1, y0, y1],
#             levels=np.linspace(0, v.max(), 25),
#             interpolation='bilinear',
#             origin='lower')
# ax.scatter(x, y, color='r', alpha=0.25, transform=transform)
ax.set_title("KDE of image sample spatial distribution")
plt.show()